<center><a href="https://5loi.com/about_loi"><img src="images/DLI_Header.png" width="400" height="186" /></a></center>

# 3. 卷积神经网络
本练习中，您将再次使用美国手语数据集训练模型。上一次我们已能对训练数据集获得很高的准确率，但模型并没有很好地泛化到验证数据集。这种无法很好地泛化到非训练数据上的行为称为*过拟合*。在本节中，我们将介绍一种流行的模型，称为[卷积神经网络](https://www.youtube.com/watch?v=x_VrgWTKkiM&vl=en)（CNN），特别适合读取图像并对其进行分类。

## 3.1 目标

在完成本节时，您将能够：
* 专门为 CNN 准备数据
* 创建更复杂的 CNN 模型，了解多种类型的模型层
* 训练 CNN 模型并观察其性能

In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

## 3.2 加载和准备数据

让我们像上一个 notebook 中那样加载 DataFrame：

In [2]:
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")

ASL 数据已经是展开的了。

In [3]:
sample_df = train_df.head().copy()  # Grab the top 5 rows
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [4]:
sample_x.shape

(5, 784)

在这种格式下，我们没有关于哪些像素彼此相邻的所有信息。因此，我们无法应用能检测特征的卷积操作。让我们用 [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) 对数据集进行纬度变换，使其变成 28x28 像素的格式。这将允许我们的卷积操作关联像素组并检测重要特征。

注意，对于我们模型的第一个卷积层，我们不仅需要图像的高度和宽度，还需要[颜色通道](https://www.photoshopessentials.com/essentials/rgb/)的数量。我们的图像是灰度的，所以只有 1 个通道。

这意味着需要将当前形状 `(5, 784)` 转换为 `(5, 1, 28, 28)`。使用 [NumPy](https://numpy.org/doc/stable/index.html) 数组，我们可以为任何我们希望保持不变的维度传递 `-1`。

In [5]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

### 3.2.2 创建数据集

让我们把上述步骤加到 `MyDataset` 类中。

#### 练习

下面的类定义中有 4 个 `FIXME`。你能把它们替换成正确的值么？

In [6]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()  # Some operations below are in-place
        y_df = x_df.pop(FIXME)
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, FIXME, FIXME, FIXME)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

#### 解答

点击下方的 `...` 查看答案。

In [7]:
# SOLUTION
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()  # Some operations below are in-place
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

### 3.2.3 创建 DataLoader

接下来，让我们从 Dataset 中创建 DataLoader。

#### 练习

下面的其中一个函数调用里少了一个 `shuffle=True` 参数。你能回忆起是哪个并把它加上去么？

In [8]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

#### 解答

点击下方的 `...` 查看答案。

In [10]:
# SOLUTION
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

下面从 DataLoader 中取出一个批次来确认数据格式。

In [11]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.5020, 0.5255, 0.5490,  ..., 0.6627, 0.6510, 0.6471],
           [0.5176, 0.5412, 0.5608,  ..., 0.6784, 0.6706, 0.6667],
           [0.5255, 0.5490, 0.5647,  ..., 0.6902, 0.6863, 0.6824],
           ...,
           [0.3569, 0.3647, 0.3765,  ..., 0.7647, 0.7412, 0.7373],
           [0.3647, 0.3647, 0.3686,  ..., 0.3725, 0.5922, 0.8392],
           [0.3686, 0.3686, 0.3765,  ..., 0.4902, 0.6314, 0.7490]]],
 
 
         [[[0.6667, 0.6863, 0.7059,  ..., 0.7373, 0.5333, 0.5176],
           [0.6824, 0.7020, 0.7216,  ..., 0.7333, 0.6902, 0.4510],
           [0.6980, 0.7255, 0.7412,  ..., 0.7373, 0.7569, 0.6039],
           ...,
           [0.4706, 0.4706, 0.4745,  ..., 0.1098, 0.0627, 0.0275],
           [0.4784, 0.4784, 0.4824,  ..., 0.0863, 0.0824, 0.0235],
           [0.4784, 0.4745, 0.5020,  ..., 0.0627, 0.0863, 0.0510]]],
 
 
         [[[0.4745, 0.4863, 0.5059,  ..., 0.5451, 0.5412, 0.5294],
           [0.4784, 0.4941, 0.5137,  ..., 0.5529, 0.5451, 0.5451],
           [0.4824

它看起来有变化了，再通过 `shape` 确认一下。

In [12]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [13]:
batch[1].shape

torch.Size([32])

## 3.3 创建卷积模型

如今，许多数据科学家通过借鉴类似项目的模型配置来开始他们的项目。假设问题并非毫不相关，那么很有可能已经有人创建了表现很好的模型，并将其发布在 [TensorFlow Hub](https://www.tensorflow.org/hub) 和 [NGC Catalog](https://ngc.nvidia.com/catalog/models) 等在线仓库中。今天，我们将提供一个适用于这个问题的模型。

<img src="images/cnn.png" width=180 />

我们在讲座中介绍了许多不同类型的层，这里我们将逐一回顾它们，并提供它们的文档链接。如有疑问，请阅读官方文档（或在 [Stack Overflow](https://stackoverflow.com/) 上提问）。

In [14]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

### 3.3.1 [Conv2D](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

<img src="images/conv2d.png" width=300 />

这些是我们的 2D 卷积层。小型卷积核（kernel）将在输入图像上滑动，检测对分类重要的特征。模型早期的卷积将检测简单的特征，如线条。后面的卷积将检测更复杂的特征。让我们看看第一个 Conv2D 层:
```Python
nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1)
```
25 指的是将要学习的滤波器（filter）数量。尽管 `kernel_size = 3`，PyTorch 会假设我们想要的是 3 x 3 的滤波器。`stride` 指的是滤波器在图像上滑动时的步长。`padding` 决定了由滤波器创建的输出图像是否与输入图像的大小匹配。

### 3.3.2 [BatchNormalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)

与归一化输入类似，批量归一化通过缩放隐藏层中的值来改善训练。[在这里]((https://blog.paperspace.com/busting-the-myths-about-batch-normalization/))可以阅读更多详细信息。

关于批量归一化层应该放在哪里最好，存在一些争议。[这个 Stack Overflow 帖子](https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout)汇集了许多观点。

### 3.3.3 [MaxPool2D](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

<img src="images/maxpool2d.png" width=300 />

最大池化实质上是将图像缩小到较低的分辨率。这样做是为了帮助模型对平移（translation，物体左右移动）具有鲁棒性，同时也使我们的模型运行更快。

### 3.3.4 [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)

<img src="images/dropout.png" width=360 />

Dropout 是一种防止过拟合的技术。Dropout 随机选择一部分神经元并将其关闭，使它们在特定的前向或后向传播中不参与计算。这有助于确保网络具有鲁棒性和冗余性，不会过度依赖任何一个区域来得出结果。

### 3.3.5 [Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)

Flatten 层将一个多维的层输出展平成一维数组。这个输出被称为特征向量，将连接到最终的分类层。

### 3.3.6 [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

我们在早期的模型中已经见过全连接的线性层。我们的第一个全连接层（512个单元）以特征向量为输入，学习哪些特征将对特定分类有贡献。第二个全连接层（24个单元）是最终的分类层，输出我们的预测结果。

## 3.4 总结模型

您可能感觉信息量有点大，但不用担心。现在不需要完全理解所有内容就能有效地训练卷积模型。最重要的是我们知道了它可以帮助从图像中提取有用信息，并可以用于分类任务。

In [15]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

由于我们试图解决的问题仍然相同（分类ASL图像），我们将继续使用相同的`损失函数`和`准确率`指标。

In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [17]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

## 3.5 训练模型

尽管模型架构差别很大，但训练过程看上去完全一样。

#### 练习

这些是与之前相同的 `train` 和 `validate` 函数，但它们被混在一起了。你能正确命名每个函数并替换 `FIXME` 吗？

#### 解答

点击下方的 `...` 查看答案。

In [18]:
# SOLUTION
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [19]:
# SOLUTION
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [20]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 263.0673 Accuracy: 0.9101
Valid - Loss: 25.3139 Accuracy: 0.9564
Epoch: 1
Train - Loss: 14.8527 Accuracy: 0.9958
Valid - Loss: 14.5266 Accuracy: 0.9774
Epoch: 2
Train - Loss: 15.0548 Accuracy: 0.9949
Valid - Loss: 18.2880 Accuracy: 0.9736
Epoch: 3
Train - Loss: 6.3722 Accuracy: 0.9981
Valid - Loss: 6.9258 Accuracy: 0.9922
Epoch: 4
Train - Loss: 13.2297 Accuracy: 0.9958
Valid - Loss: 18.2248 Accuracy: 0.9791
Epoch: 5
Train - Loss: 0.5619 Accuracy: 0.9999
Valid - Loss: 10.3425 Accuracy: 0.9824
Epoch: 6
Train - Loss: 15.8552 Accuracy: 0.9952
Valid - Loss: 14.1589 Accuracy: 0.9819
Epoch: 7
Train - Loss: 7.1374 Accuracy: 0.9973
Valid - Loss: 22.8486 Accuracy: 0.9636
Epoch: 8
Train - Loss: 2.6616 Accuracy: 0.9990
Valid - Loss: 23.7614 Accuracy: 0.9664
Epoch: 9
Train - Loss: 9.1858 Accuracy: 0.9973
Valid - Loss: 14.9869 Accuracy: 0.9721
Epoch: 10
Train - Loss: 1.5486 Accuracy: 0.9994
Valid - Loss: 16.3305 Accuracy: 0.9770
Epoch: 11
Train - Loss: 8.4807 Accuracy: 0.9976


### 3.5.1 结果讨论
看起来大有改善！训练准确率非常高，且验证准确率也已得到提升。这是一个很棒的结果，因为我们所做的就是换了一个新模型。

您可能还会看到验证准确率有所波动，这表明我们的模型的泛化能力还有改善余地。好在我们还有别的措施供我们使用，下一讲中我们继续讨论。

## 3.6 总结

在本节中，您利用了几种新的层来实现 CNN，其表现比上一节中使用的简单的模型更好。希望您对使用准备好的数据创建和训练模型的整个过程更加熟悉。

### 3.6.1 清理显存
继续后面的内容前，请执行以下单元清理 GPU 显存。转至下一 notebook 之前需要执行此操作。

In [21]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### 3.6.2 下一步

在前面的几节中，您专注于模型的创建和训练。为了进一步提高性能，您的注意力将转移到*数据增强*，这是一组技术的集合，可以使您的模型在比原来更多更好的可用数据上进行训练。